# Selección de características

Una vez extraidas las características hay que analizarlas y seleccionar aquellas que consideremos útiles

## Imports necesarios

In [57]:
import os
import pickle
from collections import defaultdict, Counter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\franp\anaconda3\envs\TFG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Funciones que se van a usar

Función para importar estructuras de datos de ficheros .pkl

In [38]:
def loadCharacteristics(path):
    with open(path, "rb") as f:
        return pickle.load(f)

Función para guardar datos como .pkl

In [3]:
def saveCharacteristics(path, data):
    with open(path, "wb") as file:
        pickle.dump(data, file)

Función para guardar un diccionario de características como un fichero .txt

In [4]:
def saveCharacteristicsAsTxt(path, dataCounter):
    with open(path, "w", encoding = "utf-8") as file:
        for key in dataCounter:
            file.write(str(key) + ": ")

            for subkey in dataCounter[key]:
                file.write("(" + str(subkey) + ", " + str(dataCounter[key][subkey]) + ")" )
            
            file.write("\n")

## Selección de sustantivos

In [44]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/5. Filtered Antonyms Good"

nouns = {}

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    for noun in characteristics.keys():
        if noun in nouns:
            nouns[noun] += 1
        else:
            nouns[noun] = 1

In [46]:
nounsOrdenado = dict(sorted(nouns.items(), key=lambda x: x[1], reverse=True))
nounsOrdenado

{'estacion': 245,
 'metro': 171,
 'escalera': 158,
 'movilidad': 147,
 'acceso': 138,
 'servicio': 106,
 'linea': 103,
 'cobertura': 74,
 'zona': 73,
 'maquina': 65,
 'conexion': 63,
 'salida': 60,
 'senalizacion': 48,
 'personal': 45,
 'instalacion': 45,
 'frecuencia': 40,
 'ascensor': 38,
 'tren': 37,
 'gente': 36,
 'estado': 35,
 'informacion': 34,
 'experiencia': 34,
 'transporte': 33,
 'persona': 32,
 'limpieza': 32,
 'diseno': 30,
 'autobus': 30,
 'aire': 30,
 'vestibulo': 30,
 'anden': 28,
 'horario': 26,
 'barrio': 26,
 'calle': 26,
 'hora': 26,
 'verde': 26,
 'atencion': 25,
 'comunicacion': 24,
 'problema': 23,
 'red': 23,
 'seguridad': 22,
 'super': 22,
 'plaza': 22,
 'opinion': 21,
 'bano': 20,
 'dia': 20,
 'centro': 20,
 'tiempo': 20,
 'panel': 19,
 'usuarios': 18,
 'limpio': 18,
 'puerta': 17,
 'intercambiador': 17,
 'senora': 17,
 'maquinas': 17,
 'abono': 17,
 'entorno': 16,
 'moderna': 16,
 'olor': 15,
 'cartel': 15,
 'tarjeta': 15,
 'falta': 15,
 'forma': 14,
 'arquit

In [7]:
len(nouns.keys())

1487

Como se puede ver, hay muchos sustantivos, pero la mayoría no son relevantes. Además, aquellos que no son representatitvos no tienen mucha frecuencia (número de estaciones en las que aparece).

Se va a guardar esta información en un fichero llamado nouns.txt

In [8]:
with open(os.path.join(inputFolderPath, "nouns.txt"), "w", encoding = "utf-8") as file:
    for noun in nouns.keys():
        file.write(noun + ": " + str(nouns[noun]) + "\n")

In [48]:
limit = 30
counter = 0

for noun in nouns.keys():
    if nouns[noun] > limit:
        counter += 1

print("sustantivos con una frecuencia mayor a " + str(limit) + ": " + str(counter))

sustantivos con una frecuencia mayor a 30: 25


Se van a ver cuales son esos sustantivos

In [49]:
filteredNouns = {}

for noun in nouns.keys():
    if nouns[noun] > limit:
        filteredNouns[noun] = nouns[noun]

filteredNouns

{'acceso': 138,
 'movilidad': 147,
 'cobertura': 74,
 'estacion': 245,
 'senalizacion': 48,
 'maquina': 65,
 'escalera': 158,
 'salida': 60,
 'metro': 171,
 'servicio': 106,
 'personal': 45,
 'informacion': 34,
 'conexion': 63,
 'zona': 73,
 'instalacion': 45,
 'ascensor': 38,
 'estado': 35,
 'tren': 37,
 'linea': 103,
 'gente': 36,
 'transporte': 33,
 'experiencia': 34,
 'persona': 32,
 'limpieza': 32,
 'frecuencia': 40}

Únicamente vamos a seleccionar los sustantivos que aparezcan en más de 30 estaciones 

In [11]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/3. Filtered Antonyms"
outputFolderPath = "../1. Data/6. Reviews Characteristics/4. Filtered Nouns"

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    # Filter the selected nouns 
    finalCharacteristics = {}
    for noun in characteristics.keys():
        if noun in filteredNouns.keys():
            finalCharacteristics[noun] = characteristics[noun]
    
    # Save the data
    station = station[:-4]

    outputPathPkl = os.path.join(outputFolderPath, station + ".pkl")
    outputPathTxt = os.path.join(outputFolderPath, station + ".txt")

    saveCharacteristics(outputPathPkl, Counter(finalCharacteristics))
    saveCharacteristicsAsTxt(outputPathTxt, Counter(finalCharacteristics))
    
    

## Selección de caracteristicas

Se procede a repetir el mismo proceso pero con cada una de las caracteristicas de los sustantivos

In [42]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/5. Filtered Antonyms Good"

adjetives = {}

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    for adjs in characteristics.values():
        for adj, freq in adjs.items():
            if adj in adjetives:
                adjetives[adj] += freq
            else:
                adjetives[adj] = freq

In [54]:
adjetivesOrdenado = dict(sorted(adjetives.items(), key=lambda x: x[1], reverse=True))
adjetivesOrdenado

{'buen': 912,
 'limpio': 454,
 'mecánico': 382,
 'reducido': 298,
 'importante': 273,
 'gran': 264,
 'moderno': 230,
 'grande': 225,
 'excelente': 211,
 'amplio': 197,
 'accesible': 184,
 'bien comunicado': 183,
 'mejor': 171,
 'público': 169,
 'mucho bonito': 159,
 'mucho bien': 154,
 'bien': 149,
 'mucho buen': 149,
 'fácil': 145,
 'normal': 143,
 'antiguo': 139,
 'bonito': 136,
 'comercial': 136,
 'mucho limpio': 130,
 'situado': 128,
 'nuevo': 125,
 'ascensor': 121,
 'parada': 116,
 'único': 110,
 'cómodo': 107,
 'pequeño': 100,
 'bastante bien': 99,
 'seguro': 92,
 'mayor': 89,
 'intercambiador': 89,
 'rápido': 88,
 'enorme': 84,
 'junto': 82,
 'primero': 82,
 'móvil': 80,
 'residencial': 77,
 'claro': 77,
 'ubicado': 77,
 'ligero': 76,
 'buena': 75,
 'tranquilo': 74,
 'solo': 71,
 'bien señalizado': 70,
 'bien cuidado': 70,
 'expendedor': 67,
 'peor': 65,
 'mucho concurrido': 61,
 'malo': 58,
 'adaptado': 58,
 'personal': 57,
 'principal': 57,
 'bastante grande': 57,
 'mucho ampl

In [55]:
len(adjetivesOrdenado.keys())

3213

Al igual que con los sustantivos, hay muchos adjetivos, pero la mayoría no son relevantes. Además, aquellos que no son representatitvos no tienen mucha frecuencia. Por lo que vamos a filtrarlos por un cierto limite.

In [52]:
aux = {}

for adj in adjetives.keys():
    if adjetives[adj] >= limit:
        aux[adj] = adjetives[adj]

filteredAdjetives = dict(sorted(aux.items(), key=lambda x: x[1], reverse=True))
filteredAdjetives

{'buen': 912,
 'limpio': 454,
 'mecánico': 382,
 'reducido': 298,
 'importante': 273,
 'gran': 264,
 'moderno': 230,
 'grande': 225,
 'excelente': 211,
 'amplio': 197,
 'accesible': 184,
 'bien comunicado': 183,
 'mejor': 171,
 'público': 169,
 'mucho bonito': 159,
 'mucho bien': 154,
 'bien': 149,
 'mucho buen': 149,
 'fácil': 145,
 'normal': 143,
 'antiguo': 139,
 'bonito': 136,
 'comercial': 136,
 'mucho limpio': 130,
 'situado': 128,
 'nuevo': 125,
 'ascensor': 121,
 'parada': 116,
 'único': 110,
 'cómodo': 107,
 'pequeño': 100,
 'bastante bien': 99,
 'seguro': 92,
 'mayor': 89,
 'intercambiador': 89,
 'rápido': 88,
 'enorme': 84,
 'junto': 82,
 'primero': 82,
 'móvil': 80,
 'residencial': 77,
 'claro': 77,
 'ubicado': 77,
 'ligero': 76,
 'buena': 75,
 'tranquilo': 74,
 'solo': 71,
 'bien señalizado': 70,
 'bien cuidado': 70,
 'expendedor': 67,
 'peor': 65,
 'mucho concurrido': 61,
 'malo': 58,
 'adaptado': 58,
 'personal': 57,
 'principal': 57,
 'bastante grande': 57,
 'mucho ampl

In [56]:
len(filteredAdjetives.keys())

121

In [58]:
# Cargar modelo multilingüe
model1 = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
model2 = SentenceTransformer("sentence-transformers/LaBSE")

def areSynonyms(palabra1, palabra2, umbral=0.65):
    emb1 = model1.encode([palabra1])
    emb2 = model1.encode([palabra2])
    emb11 = model2.encode([palabra1])
    emb22 = model2.encode([palabra2])
    sim1 = cosine_similarity(emb1, emb2)[0][0]
    sim2 = cosine_similarity(emb11, emb22)[0][0]
    sim = ((sim1 + sim2) / 2)
    return  sim >= umbral

In [64]:
filteredSynonymsAdj = {}
viewed = set()
for adj1 in filteredAdjetives:
    for adj2 in filteredAdjetives:
        if adj1 == adj2 or adj2 in viewed:
            continue
        if areSynonyms(adj1,adj2):
            filteredSynonymsAdj[adj1] = filteredAdjetives[adj1] + filteredAdjetives[adj2]
            viewed.add(adj2)
    viewed.add(adj1)

In [65]:
filteredSynonymsAdj

{'buen': 943,
 'limpio': 484,
 'reducido': 328,
 'importante': 303,
 'gran': 298,
 'moderno': 269,
 'excelente': 241,
 'amplio': 230,
 'accesible': 236,
 'bien comunicado': 226,
 'mejor': 236,
 'mucho bonito': 194,
 'fácil': 221,
 'bonito': 171,
 'situado': 160,
 'nuevo': 235,
 'único': 154,
 'seguro': 137,
 'mayor': 134,
 'intercambiador': 156,
 'junto': 112,
 'adaptado': 91,
 'principal': 88,
 'disponible': 86,
 'conveniente': 83,
 'eficiente': 73}

In [18]:
import re
import pickle
from collections import defaultdict, Counter

def parse_txt_to_defaultdict_counter(path_txt):
    result = defaultdict(Counter)
    with open(path_txt, "r", encoding="utf-8") as f:
        for line in f:
            if ':' not in line:
                continue
            noun, values_raw = line.split(":", 1)
            noun = noun.strip()
            tuplas = re.findall(r'\(([^,]+),\s*(\d+)\)', values_raw)
            for adj, freq in tuplas:
                result[noun][adj.strip()] += int(freq)
    return result

In [36]:
def saveCharacteristics(path, data):
    with open(path, "wb") as file:
        pickle.dump(data, file)

In [37]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/3. Filtered Antonyms"
outputFolderPath = "../1. Data/6. Reviews Characteristics/5. Filtered Antonyms Good"

# Dictionary with the characteristics of all the stations
data = {}

adjectives = {}

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".txt"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = parse_txt_to_defaultdict_counter(inputPath)

    # Add it to the dictionary with all the information
    data[station[:-4]] = characteristics

    # Save the processed characteristics
    outputPath = os.path.join(outputFolderPath, station[:-4])

    saveCharacteristics(outputPath + ".pkl", characteristics)

In [29]:
def parse_txt_to_defaultdict_counter(path_txt):
    result = defaultdict(Counter)
    with open(path_txt, "r", encoding="utf-8") as f:
        for line in f:
            if ':' not in line:
                continue
            noun, values_raw = line.split(":", 1)
            noun = noun.strip()
            tuplas = re.findall(r'\(([^,]+),\s*(\d+)\)', values_raw)
            for adj, freq in tuplas:
                result[noun][adj.strip()] += int(freq)
    return result

In [39]:
aux = loadCharacteristics("../1. Data/6. Reviews Characteristics/5. Filtered Antonyms Good/Abrantes.pkl")

In [40]:
aux

defaultdict(collections.Counter,
            {'acceso': Counter({'fácil': 1}),
             'movilidad': Counter({'reducido': 1}),
             'cobertura': Counter({'móvil': 1}),
             'rollo': Counter({'mal': 1}),
             'estacion': Counter({'excelente': 2, 'mucho tranquilo': 1}),
             'trabajador': Counter({'mucho amargado': 1}),
             'olor': Counter({'mal': 1})})